### 1. Создание признакового пространства 

Алгоритмы машинного обучения не могут напрямую работать с сырым текстом, поэтому необходимо конвертировать текст в наборы цифр (векторы). Это называется извлечением признаков.

##### Мешок слов
– это популярная и простая техника извлечения признаков, используемая при работе с текстом. Она описывает вхождения каждого слова в текст.

Чтобы использовать модель, нам нужно:

- Определить словарь известных слов (токенов).
- Выбрать степень присутствия известных слов.

Любая информация о порядке или структуре слов игнорируется. Вот почему это называется МЕШКОМ слов. Эта модель пытается понять, встречается ли знакомое слово в документе, но не знает, где именно оно встречается.

Интуиция подсказывает, что схожие документы имеют схожее содержимое. Также, благодаря содержимому, мы можем узнать кое-что о смысле документа.

Пример:
Рассмотрим шаги создания этой модели. Мы используем только 4 предложения, чтобы понять, как работает модель. В реальной жизни вы столкнетесь с бОльшими объемами данных.

In [2]:
documents = ["I like this movie, it's it's it's funny.", 'I hate this movie.', 'This was awesome! I like it.', 'Nice one. I love it.']

Определяем словарь и создаем векторы документа. Соберем все уникальные слова из 4 загруженных предложений, игнорируя регистр, пунктуацию и односимвольные токены. Это и будет наш словарь (известные слова).

Для создания словаря можно использовать класс CountVectorizer из библиотеки sklearn.

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
documents

["I like this movie, it's it's it's funny.",
 'I hate this movie.',
 'This was awesome! I like it.',
 'Nice one. I love it.']

In [6]:
CountVectorizer?

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

count_vectorizer = CountVectorizer(ngram_range=[1,3], analyzer='char', binary=False, tokenizer=str.split)

# Создаем the Bag-of-Words модель
bag_of_words = count_vectorizer.fit_transform(documents)

# Отобразим Bag-of-Words модель как DataFrame
feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(bag_of_words.toarray(), columns = feature_names)

,,a,aw,f,fu,h,ha,i,i,it,...,ve,vi,vie,w,wa,was,we,wes,y,y.
0,7,0,0,1,1,0,0,3,0,3,...,0,1,1,0,0,0,0,0,1,1
1,3,0,0,0,0,1,1,0,0,0,...,0,1,1,0,0,0,0,0,0,0
2,5,1,1,0,0,0,0,2,1,1,...,0,0,0,2,1,1,1,1,0,0
3,4,0,0,0,0,0,0,2,1,1,...,1,0,0,0,0,0,0,0,0,0


Когда размер словаря увеличивается, вектор документа тоже растет. В примере выше, длина вектора равна количеству известных слов.

В некоторых случаях, у нас может быть неимоверно большой объем данных и тогда вектор может состоять из тысяч или миллионов элементов. Более того, каждый документ может содержать лишь малую часть слов из словаря.

Как следствие, в векторном представлении будет много нулей. Векторы с большим количеством нулей называются разреженным векторами (sparse vectors), они требуют больше памяти и вычислительных ресурсов. Частично эту проблему можно реить хорошей предобработкой.


##### Мешок N-грамм

Другой, более сложный способ создания словаря – использовать сгруппированные слова. Это изменит размер словаря и даст мешку слов больше деталей о документе. Такой подход называется «N-грамма».

N-грамма это последовательность каких-либо сущностей (слов, букв, чисел, цифр и т.д.). В контексте языковых корпусов, под N-граммой обычно понимают последовательность слов. Юниграмма это одно слово, биграмма это последовательность двух слов, триграмма – три слова и так далее. Цифра N обозначает, сколько сгруппированных слов входит в N-грамму. В модель попадают не все возможные N-граммы, а только те, что фигурируют в корпусе.

Пример:
Рассмотрим такое предложение:The office building is open today

Вот его биграммы:

- the office
- office building
- building is
- is open
- open today

Как видно, мешок биграмм – это более действенный подход, чем мешок слов.

In [21]:
from nltk.util import ngrams

text = "I like this movie, it's funny. I hate this movie. This was awesome! I like it. Nice one. I love it."
tokenized = text.split()
bigrams = ngrams(tokenized, (2))
list(bigrams)


[('I', 'like'),
 ('like', 'this'),
 ('this', 'movie,'),
 ('movie,', "it's"),
 ("it's", 'funny.'),
 ('funny.', 'I'),
 ('I', 'hate'),
 ('hate', 'this'),
 ('this', 'movie.'),
 ('movie.', 'This'),
 ('This', 'was'),
 ('was', 'awesome!'),
 ('awesome!', 'I'),
 ('I', 'like'),
 ('like', 'it.'),
 ('it.', 'Nice'),
 ('Nice', 'one.'),
 ('one.', 'I'),
 ('I', 'love'),
 ('love', 'it.')]

##### TF-IDF

У частотного скоринга есть проблема: слова с наибольшей частотностью имеют, соответственно, наибольшую оценку. В этих словах может быть не так много информационного выигрыша для модели, как в менее частых словах. Один из способов исправить ситуацию – понижать оценку слова, которое часто встречается во всех схожих документах. Это называется TF-IDF.

TF-IDF (сокращение от term frequency — inverse document frequency) – это статистическая мера для оценки важности слова в документе, который является частью коллекции или корпуса.

Скоринг по TF-IDF растет пропорционально частоте появления слова в документе, но это компенсируется количеством документов, содержащих это слово.

<img src='image/tf_idf.PNG'>

In [31]:
TfidfVectorizer?

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

document = ["I like this movie, it's funny funny.", 'I hate this movie.', 'This was awesome! I like it.', 'Nice one. I love it.']
tfidf_vectorizer = TfidfVectorizer()
values = tfidf_vectorizer.fit_transform(document)

# Show the Model as a pandas DataFrame
feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(values.toarray(), columns = feature_names)

,awesome,funny,hate,it,like,love,movie,nice,one,this,was
0,0.000000,0.812578,0.000000,0.259329,0.320323,0.000000,0.320323,0.000000,0.000000,0.259329,0.000000
1,0.000000,0.000000,0.702035,0.000000,0.000000,0.000000,0.553492,0.000000,0.000000,0.448100,0.000000
2,0.539445,0.000000,0.000000,0.344321,0.425305,0.000000,0.000000,0.000000,0.000000,0.344321,0.539445
3,0.000000,0.000000,0.000000,0.345783,0.000000,0.541736,0.000000,0.541736,0.541736,0.000000,0.000000


In [29]:
tfidf_vectorizer.idf_

array([1.91629073, 1.91629073, 1.91629073, 1.22314355, 1.51082562,
       1.91629073, 1.51082562, 1.91629073, 1.91629073, 1.22314355,
       1.91629073])

##### HashingVectorizer

Подсчеты и частоты могут быть очень полезны, но одним из ограничений этих методов является то, что словарный запас может стать очень большим. Это, в свою очередь, потребует больших векторов для кодирования документов и налагает большие требования к памяти и замедляет алгоритмы.

Умный обходной путь - использовать односторонний хэш слов, чтобы преобразовать их в целые числа. Умная часть заключается в том, что словарь не требуется, и вы можете выбрать произвольный длинный вектор фиксированной длины. Недостатком является то, что хеш является односторонней функцией, поэтому нет способа преобразовать кодировку обратно в слово (что может не иметь значения для многих контролируемых задач обучения).HashingVectorizer класс реализует этот подход, который можно использовать для последовательного хеширования слов, а затем для токенизации и кодирования документов по мере необходимости.

Пример ниже демонстрирует HashingVectorizer для кодирования одного документа.

In [33]:
HashingVectorizer?

In [32]:
from sklearn.feature_extraction.text import HashingVectorizer

document = ["I like this movie, it's funny.", 'I hate this movie.', 'This was awesome! I like it.', 'Nice one. I love it.']
vectorizer = HashingVectorizer(n_features=2**4,)
values = vectorizer.fit_transform(document)
print(values.shape)
print(values.toarray())

(4, 16)
[[ 0.37796447  0.          0.          0.          0.          0.
   0.         -0.37796447  0.          0.         -0.37796447  0.
   0.          0.          0.75592895  0.        ]
 [ 0.          0.          0.         -0.57735027  0.          0.
   0.          0.          0.          0.         -0.57735027  0.
   0.          0.          0.57735027  0.        ]
 [ 0.          0.4472136   0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.89442719  0.        ]
 [ 0.          0.          0.          0.         -0.5         0.
   0.          0.          0.         -0.5         0.          0.
   0.          0.5         0.5         0.        ]]


Выполнение примера кодирует образец документа как разреженный массив из 16 элементов. Значения закодированного документа соответствуют нормализованному количеству слов по умолчанию в диапазоне от -1 до 1, но могут быть сделаны простые целочисленные счетчики путем изменения конфигурации по умолчанию.

##### Для чего нужны Vectorizers?

Алгоритмы машинного обучения не могут напрямую работать с сырым текстом, поэтому необходимо конвертировать текст в наборы цифр (векторы). Уже с векторным представлением можно производить классификацию, к примеру.

In [8]:
# Загружаем данные
data = open('corpus').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append(content[0])
    texts.append(" ".join(content[1:]))

# создаем df
trainDF = pd.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels
trainDF.head(5)

,text,label
0,Stuning even for the non-gamer: This sound tra...,__label__2
1,The best soundtrack ever to anything.: I'm rea...,__label__2
2,Amazing!: This soundtrack is my favorite music...,__label__2
3,Excellent Soundtrack: I truly like this soundt...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After He...",__label__2


In [9]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)


count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

classifier = linear_model.LogisticRegression()
classifier.fit(xtrain_count, train_y)
predictions = classifier.predict(xvalid_count)
predictions

array([0, 1, 0, ..., 0, 0, 1])

In [10]:
xtrain_count.shape

(7500, 31666)

In [11]:
from sklearn.metrics import accuracy_score

In [12]:
accuracy_score(valid_y, predictions)

0.8728

In [13]:
trainDF['label'].value_counts()

__label__1    5097
__label__2    4903
Name: label, dtype: int64

In [22]:
vct = HashingVectorizer(analyzer='word', token_pattern=r'\w{1,}', n_features=200)
vct.fit(trainDF['text'])

xtrain_hash =  vct.transform(train_x)
xvalid_hash =  vct.transform(valid_x)

classifier = linear_model.LogisticRegression()
classifier.fit(xtrain_hash, train_y)
predictions = classifier.predict(xvalid_hash)
predictions

array([0, 1, 1, ..., 1, 1, 1])

In [23]:
accuracy_score(valid_y, predictions)

0.7224

In [24]:
xtrain_hash.shape

(7500, 200)

## NN

In [25]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Conv1D, GRU, LSTM, Dropout
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [26]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

In [27]:
train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [28]:
for raw in train_data.take(1):
    pass

In [29]:
raw

(<tf.Tensor: shape=(16,), dtype=string, numpy=
 array([b'Soooooooooooo NOT Funny: This movie was not only NOT funny, but it was excruciatingly boring. I think Adam was channeling the Aristocrats. Whilst Aristocrats was not as funny as it thought itself to be, it was still better than this garbage.',
        b'Pretty scary-not so swift ending: I think that The Haunting is a good scary movie(as opposed to movies like "Scream"). Instead of dealing with pyscho people running around in masks, it actually deals with the supernatural which is always fascinating. The movie was cast very well and the special effects are incredible. The ending however, twisted the movie into a theme of bliss, contentment, and nurturing. A good scary movie should leave you with a sense of "what will happen next?". On the other hand, it would be hard to have a nightmare about it! All in all, it is a good movie that really screams the theme "good vs evil".',
        b'Another fun Rudy Maxa DVD: We have enjoyed all 

In [30]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [31]:
def custom_standardization(input_data):
    
    return input_data

vocab_size = 10000
seq_len = 100

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [34]:
embedding_dim=200

model = Sequential([
    vectorize_layer,
    Embedding(vocab_size, embedding_dim),
    Conv1D(200, (3)),
    Conv1D(200, (2)),
    GRU(300),
    #GlobalAveragePooling1D(),
    Dense(100, activation='relu'),
    Dense(1)
])

In [35]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [36]:
model.fit(train_data, validation_data=valid_data, epochs=15)

Epoch 1/15
469/469 [==============================] - 81s 166ms/step - loss: 0.6974 - accuracy: 0.5112 - val_loss: 0.6932 - val_accuracy: 0.4992
Epoch 2/15
111/469 [======>.......................] - ETA: 53s - loss: 0.6882 - accuracy: 0.5079

KeyboardInterrupt: 

In [121]:
class myNet(tf.keras.Model):
    def __init__(self):
        super(myNet, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim, name="embedding")
        self.conv1 = Conv1D(200, (3))
        self.conv2 = Conv1D(200, (3))
        self.gPool = GlobalAveragePooling1D()
        self.fc1 = Dense(100, activation='relu')
        self.fc2 = Dense(1)


    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        x1 = self.conv1(x)
        x = self.conv2(x)
        x = self.gPool(x)
        x = self.fc1(x)
        x = self.ss(x)
        return self.fc2(x)

In [122]:
mmodel = myNet()

In [123]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [124]:
mmodel.fit(train_data, validation_data=valid_data, epochs=15)

Epoch 1/15
469/469 [==============================] - 13s 25ms/step - loss: 0.5852 - accuracy: 0.6378 - val_loss: 0.4185 - val_accuracy: 0.8160
Epoch 2/15
469/469 [==============================] - 13s 27ms/step - loss: 0.2444 - accuracy: 0.9018 - val_loss: 0.4274 - val_accuracy: 0.8320
Epoch 3/15
469/469 [==============================] - 13s 28ms/step - loss: 0.1097 - accuracy: 0.9640 - val_loss: 0.5540 - val_accuracy: 0.7940
Epoch 4/15
469/469 [==============================] - 12s 26ms/step - loss: 0.0759 - accuracy: 0.9690 - val_loss: 0.7106 - val_accuracy: 0.8244
Epoch 5/15
469/469 [==============================] - 11s 24ms/step - loss: 0.0567 - accuracy: 0.9782 - val_loss: 0.7463 - val_accuracy: 0.8212
Epoch 6/15
469/469 [==============================] - 11s 24ms/step - loss: 0.0328 - accuracy: 0.9883 - val_loss: 0.9834 - val_accuracy: 0.8168
Epoch 7/15
469/469 [==============================] - 12s 26ms/step - loss: 0.0224 - accuracy: 0.9909 - val_loss: 1.3219 - val_accuracy: